# Instalasi Library

In [ ]:
!pip install -U -q google-generativeai langchain langchain-google-genai langchain_community pypdf chromadb

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from IPython.display import Markdown as md

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import userdata

In [ ]:
# import google.generativeai as genai

# masukan API key kalian
api_key = userdata.get('GEMINI')
# genai.configure(api_key=api_key)

In [ ]:
chat_model = ChatGoogleGenerativeAI(google_api_key=api_key,
                                   model="gemini-2.5-flash")

In [ ]:
# download pdf dengan curl
!curl -o  ai_pv.pdf https://www.pearsonvue.com/content/dam/VUE/vue/en/documents/clients/it-specialist/its-od-307-artificial-intel-pearson.pdf

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("ai_pv.pdf")
pages = loader.load_and_split()

In [ ]:
pages

In [ ]:
pages[0].page_content

In [ ]:
len(pages)

# Chunk

In [ ]:
import nltk
nltk.download('punkt_tab')

Agar lebih terbanyang tentang penerapan chunk, kita akan uji coba terlebih dahulu dengan text sederhana.

In [ ]:
# from langchain_text_splitters import NLTKTextSplitter
from langchain_text_splitters import CharacterTextSplitter

# Misalkan kamu memiliki dokumen seperti berikut
simple_doc = """halo nama sayaaaaaa sardi irfansyah, saya lahir di jakarta. Saya irfan. tinggal di Jakarta. saya senang belahjar math 123 456 789"""
print('panjang total karakter:',len(simple_doc),'\n')

In [ ]:
# Membuat objek NLTKTextSplitter dengan ukuran chunk dan overlap
text_splitter = CharacterTextSplitter(separator= " ", chunk_size=15, chunk_overlap=10) #default separator='\n\n'

In [ ]:
# Memecah dokumen menjadi beberapa chunk
chunks = text_splitter.split_text(simple_doc)
print(chunks,'\n')

In [ ]:
# Menampilkan hasil chunk
for i, chunk in enumerate(chunks):
    #panjang karakter
    print(f"Panjang chunk {i+1}: {len(chunk)} karakter")
    print(f"Chunk {i+1}:")
    print(chunk)
    print("-" * 50)

Penjelasan:
- Dapat kita lihat bahwa `NLTKTextSplitter` akan mencoba membuat text tersebut dipisahkan berdasarkan kalimat atau tanda `titik`. Jadi setiap ada titik maka akan dibuat chunk.
- Ketika panjang karakter lebih dari `chunk_size`, ini akan mengakibatkan peringatan warning.
- `chunk_overlap` digunakan untuk menentukan jumlah karakter yang harus tumpang tindih antara chunk yang berdekatan.

Jika anda ingin melihat ilustrasi tentang konfigurasi chunk, anda dapat melihatnya [di sini](https://dev.to/peterabel/what-chunk-size-and-chunk-overlap-should-you-use-4338).

Selanjutnya kita akan coba terapkan proses pemisahan kalimat ke dokumen yang kita gunakan.

In [ ]:
from langchain_text_splitters import NLTKTextSplitter

text_splitter = NLTKTextSplitter(chunk_size=500, chunk_overlap=100)

chunks = text_splitter.split_documents(pages)

print(len(chunks))

print(type(chunks[0]))

In [ ]:
# Mengecek hasil pemecahan menjadi chunk
print(f"Jumlah chunks yang dihasilkan: {len(chunks)}")
print(f"Tipe data chunk pertama: {type(chunks[0])}")

In [ ]:
print(len('and the problem'))

In [ ]:
# Menampilkan chunk pertama
# panjang chunk
print(f"Panjang chunk pertama: {len(chunks[0].page_content)} karakter")
print("\nContoh chunk pertama:")
print(chunks[0].page_content)  # Memastikan bahwa setiap chunk memiliki konten

# Embedding

Membuat Embedding setelah melakukan Chunking.

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding_model = GoogleGenerativeAIEmbeddings(google_api_key=api_key, model="models/embedding-001")

# Vector Databases

Cek dokumentasi [vector store](https://python.langchain.com/docs/how_to/vectorstore_retriever/).

In [ ]:
from langchain_community.vectorstores import Chroma

# Sematkan setiap chunk dan muat ke dalam chromadb
db = Chroma.from_documents(chunks, embedding_model, persist_directory="./chroma_db_")

# Menyimpan perubahan ke disk
db.persist()

In [ ]:
# mengatur koneksi untuk menghubungkan ke ChromaDB
db_connection = Chroma(persist_directory="./chroma_db_", embedding_function=embedding_model)

In [ ]:
# Mengonversi koneksi Chroma menjadi objek retriever untuk pencarian dokumen berbasis vektor
# RAG --> retrieval augmented generation

retriever = db_connection.as_retriever(search_kwargs={"k": 5})

print(type(retriever))

`k` adalah parameter yang digunakan dalam fungsi as_retriever() untuk menentukan jumlah hasil pencarian teratas (top-k) yang akan diambil dari basis data vektor (Chroma database).

Parameter ini digunakan dalam proses pencarian dokumen relevan yang mengacu pada jumlah dokumen atau chunk yang akan diambil dari kumpulan dokumen yang lebih besar (misalnya, dari Chroma vector store) berdasarkan kesamaan atau kedekatannya dengan pertanyaan yang diajukan.

> Nilai k umumnya lebih kecil dari total chunk.



In [ ]:
check_response = retriever.invoke("jelaskan sikll yang dibutuhkan untuk sertifikasi pearson vue dalam bidang AI?")
len(check_response)

In [ ]:
md(check_response[0].page_content)

# Prompt

Dokumnetasi [Prompt templates](https://python.langchain.com/docs/concepts/prompt_templates/).

In [ ]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

In [ ]:
# Membuat template pesan untuk sistem dan pesan pengguna
chat_template = ChatPromptTemplate.from_messages([
    # System Message Prompt Template
    SystemMessage(content="""Anda adalah AI yang dapat menjawab pertanyaan berdasarkan konteks dan pertanyaan dari user.
                 Anda harus menjawab pertanyaan user, berdasarkan konteks. Jawab menggunakan bahasa dari pertanyaan user.
                 Jika pertanyaan user dalam bahasa indonesia, jawab pakai bahasa indonesia, jika dalam english jawab english, jika bahasa spanyol, jawab dalam spanyol"""),

    # Human Message Prompt Template
    HumanMessagePromptTemplate.from_template("""Jawab pertanyaan berikut berdasarkan konteks.
    Jawab menggunakan bahasa dari pertanyaan user.
    Jika pertanyaan user dalam bahasa indonesia, jawab pakai bahasa indonesia, jika dalam english jawab english, jika bahasa spanyol, jawab dalam spanyol
    konteks: {context}
    pertanyaan: {question}
    jawaban: """)
])

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
# Setelah dokumen diambil oleh retriever dan di-format
formatted_docs = format_docs(chunks)  # Format dokumen yang dihasilkan dari text chunks

# Cetak hasil format
print("Hasil Format Docs:")
print(formatted_docs)  # Menampilkan hasil setelah dokumen diformat

In [ ]:
from langchain_core.runnables import RunnablePassthrough

#LCEL

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | chat_template
    | chat_model
    | output_parser
)

In [ ]:
response = rag_chain.invoke("""beritahu saya pemahaman apa yang dibutuhkan untuk sertifikasi IT spesialis: AI dari pearson vue""")

response

In [ ]:
md(response)

In [ ]:
response = rag_chain.invoke("""jelaskan sikll yang dibutuhkan untuk sertifikasi pearson vue dalam bidang AI?""")
md(response)

In [ ]:
response = rag_chain.invoke("""is there any cost we must pay to do the exam test ?""")
md(response)

# Contoh lain

misalkan pada contoh ini kita akan mencoba menerapkan RAG
dengan `RecursiveCharacterTextSplitter` dan `PromptTemplate`.

In [ ]:
# Instalasi dependensi yang dibutuhkan
!pip install -q langchain PyPDF2 #python-dotenv

In [ ]:
import os
import io
import PyPDF2
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
# from dotenv import load_dotenv
from google.colab import files
from IPython.display import Markdown as md

In [ ]:
# Membuka dan membaca file PDF
with open('/content/ai_pv.pdf', "rb") as file:
    pdf_reader = PyPDF2.PdfReader(file)
    pdf_pages = pdf_reader.pages

    # Mengekstrak teks dari semua halaman
    context = "\n\n".join(page.extract_text() for page in pdf_pages)

In [ ]:
context

In [ ]:
# Memecah teks menjadi potongan-potongan kecil
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_text(context)

In [ ]:
print(len(texts))

print(type(texts[0]))

In [ ]:
texts[0]

In [ ]:
# Membuat embeddings untuk potongan-potongan teks
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=api_key)

vector_index = Chroma.from_texts(texts, embeddings).as_retriever(search_kwargs={"k": 20})

In [ ]:
vector_index

In [ ]:
print(type(vector_index))

In [ ]:
# Mendapatkan pertanyaan dari pengguna
user_question = input("Tanyakan pertanyaan: ")

# Mendapatkan dokumen relevan untuk pertanyaan pengguna
docs = vector_index.get_relevant_documents(user_question)

In [ ]:
# Mendefinisikan template prompt
prompt_template = """
Jawablah pertanyaan ini dengan se-detail mungkin dari konteks yang diberikan,
pastikan untuk memberikan semua detail, jika jawaban tidak ada dalam
konteks yang diberikan cukup katakan, "jawaban tidak tersedia dalam konteks",
jangan memberikan jawaban yang salah\n\n
Konteks:\n {context}?\n
Pertanyaan: \n{question}\n
Jawaban:
"""

# Membuat prompt
prompt = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])

# Memuat QA chain
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", api_key=api_key)
chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

# Mendapatkan jawaban dari model
response = chain({"input_documents": docs, "question": user_question}, return_only_outputs=True)

# Menampilkan jawaban
print("\nJawaban:")
md(response['output_text'])